#Importing Libraries

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
import keras.layers as layers
from tensorflow.keras.layers import Dense
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Dropout
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.metrics import accuracy_score

#Importing Dataset Path

In [ ]:
train_path = '/kaggle/input/uts-deep-learning-2/ALS/asl_alphabet_train/asl_alphabet_train/'
test_path = '/kaggle/input/uts-deep-learning-2/ALS/asl_alphabet_test/asl_alphabet_test/'

In [ ]:
train_images = []
train_images_label = []

test_images = []
test_images_label = []

In [ ]:
import string
upper = list(string.ascii_uppercase)

In [ ]:
for abc in upper:
    curr_path = train_path + abc + '/'
    for i in os.listdir(curr_path):
        train_images.append(curr_path + i)
        train_images_label.append(abc)

In [ ]:
for i in os.listdir(test_path):
    test_images.append(test_path + i)
    test_images_label.append((i.split('_',1))[0])

In [ ]:
print(len(train_images))
print(len(test_images))

26000
28


In [ ]:
train_df_d = {'Images': train_images, 'Label': train_images_label}
test_df_d = {'Images': test_images, 'Label': test_images_label}

In [ ]:
train_df = pd.DataFrame(train_df_d)
test_df = pd.DataFrame(test_df_d)

In [ ]:
print(train_df.shape)
print(test_df.shape)

(26000, 2)
(28, 2)


In [ ]:
X = train_df.loc[:,'Images']
y = train_df.loc[:,'Label']

>Disini, pada dataset yang dibuat, yang disimpan bukanlah vector image, melainkan direction path atas masing-masing image.

>Nantinya, sebelum dilakukan fitting dan prediction, baru image akan di-generate dengan ImageDataGenerator.

#Splitting Dataset (Train & Validation)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 28, stratify=y)

In [ ]:
valid_df= pd.DataFrame(columns=['Images','Label'])
valid_df['Images'] = X_val
valid_df['Label'] = y_val

In [ ]:
X_test = test_df.loc[:,'Images']
y_test = test_df.loc[:,'Label']

In [ ]:
train_df= pd.DataFrame(columns=['Images','Label'])
train_df['Images'] = X_train
train_df['Label'] = y_train

>Disini saya menggunakan stratify = y agar pada validation dataset, split dataset membagi jumlah entry per class dengan sama rata.

#Augmentation

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=30, # rotation
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2]) # brightness

>Beberapa penjelasan terkait augmentasi yang saya lakukan adalah sebagai berikut:
*   Rotation_range = Saya masukkan karena berdasarkan analisa saya pada test dataset, terdapat foto tangan yang tidak tegak lurus pada sumbu vertical. Sehingga, untuk mengantisipasi hal tersebut, saya menambahkan rotation_range agar model dapat lebih robust dalam mengenali image yang miring saat ditesting.
*   Zoom_range = Dalam pengamatan sekilas, walaupun bentuk sign language yang sama, tetapi ada sebagian foto yang lebih dekat dengan kamera dibandingkan yang lain. Oleh karenanya, saya menambahkan zoom_range agar apabila pada prediction test dataset terdapat foto sign language yang agak memenuhi layar, maka tetap dapat dikenali
*   Horizontal flip = Saya menambahkan ini karena pada dataset, pada beberapa test data ada yang terbalik dari kiri ke kanan. Oleh karenanya, agar dapat menjaga akurasi, saya tambahkan sebagai True
*   Brightness range = Setelah menganalisa foto-foto yang ada pada train dataset, ditemukan ada image dengan pencerahan yang baik dan ada juga yang tidak terkena pencerahan. Oleh karenanya, saya menambahkan brightness range agar model mampu mengenai sign language yang sama pada dataset walaupun pencahayaan yang redup maupun terang.

In [ ]:
datagen2 = ImageDataGenerator()

>Saya membuat Image Data Generator yang terpisah untuk dataset test. Hal ini karena test dataset tidak boleh diubah-ubah. Sehingga, saya tidak melakukan augmentasi pada test dataset.

#Generate Dataset's Images

In [ ]:
train_generator_df = datagen.flow_from_dataframe(dataframe=train_df, 
                                              directory=train_path,
                                              x_col="Images", 
                                              y_col="Label", 
                                              class_mode="categorical", 
                                              target_size=(64, 64), 
                                              batch_size=16,
                                              rescale=1.0/255,
                                              seed=28)

Found 20800 validated image filenames belonging to 26 classes.


In [ ]:
test_generator_df = datagen.flow_from_dataframe(dataframe=test_df, 
                                              directory=test_path,
                                              x_col="Images", 
                                              y_col="Label", 
                                              class_mode="categorical", 
                                              target_size=(64, 64), 
                                              batch_size=16,
                                              rescale=1.0/255,
                                              seed=28,
                                              shuffle = False)

Found 28 validated image filenames belonging to 28 classes.


In [ ]:
valid_generator_df = datagen.flow_from_dataframe(dataframe=valid_df, 
                                              directory=train_path,
                                              x_col="Images", 
                                              y_col="Label", 
                                              class_mode="categorical", 
                                              target_size=(64, 64), 
                                              batch_size=16,
                                              rescale=1.0/255,
                                              seed=28)

Found 5200 validated image filenames belonging to 26 classes.


>Untuk setiap data, baik test train dan valid, saya lakukan normalisasi. Target size disesuaikan dengan ketentuan soal, yaitu 64 x 64. Saya input class-mode sebagai 'categorical' karena output class bersifat kategorikal dari A - Z.

#Modelling

In [ ]:
model = Sequential()
model.add(layers.Conv2D(64,kernel_size=5, input_shape=(64,64,3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model.add(layers.Conv2D(32,kernel_size=5))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model.add(layers.Conv2D(16,kernel_size=5))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model.add(layers.Flatten())
model.add(layers.Dense(26))
model.add(layers.Activation('relu'))
model.add(layers.Dense(26,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 60, 60, 64)        4864      
_________________________________________________________________
activation_4 (Activation)    (None, 60, 60, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 32)        51232     
_________________________________________________________________
activation_5 (Activation)    (None, 25, 25, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 16)         

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

>Disini saya menggunakan optimizer adam karena berdasarkan https://towardsdatascience.com/7-tips-to-choose-the-best-optimizer-47bb9c1219e#:~:text=Adam%20is%20the%20best%20among,for%20this%20type%20of%20datasets., Adam adalah salah satu optimizer terbaik.

>Saya juga menggunakan batch_size = 32 karena berdasarkan rujukan https://wandb.ai/ayush-thakur/dl-question-bank/reports/What-s-the-Optimal-Batch-Size-to-Train-a-Neural-Network---VmlldzoyMDkyNDU, size batch 32 adalah salah satu yang paling optimal bila dibandingkan dengan batch size yang lebih besar. Walaupun membutuhkan training time yang lebih lama, tetapi disini saya lebih mementingkan performa daripada DNN model yang dibuat.

In [ ]:
batch_size = 32
model.fit_generator(train_generator_df, 
                    epochs=30,  # one forward/backward pass of training data
                    steps_per_epoch = X_train.shape[0]//batch_size,  # number of images comprising of one epoch
                    validation_data = valid_generator_df,  # Or validation_data=valid_generator
                    validation_steps = X_val.shape[0]//batch_size)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
650/650 [==============================] - 127s 194ms/step - loss: 3.3321 - accuracy: 0.0434 - val_loss: 3.1970 - val_accuracy: 0.0544
Epoch 2/30
650/650 [==============================] - 93s 143ms/step - loss: 3.1094 - accuracy: 0.0742 - val_loss: 3.0074 - val_accuracy: 0.0860
Epoch 3/30
650/650 [==============================] - 76s 118ms/step - loss: 3.0064 - accuracy: 0.1018 - val_loss: 2.9002 - val_accuracy: 0.1292
Epoch 4/30
650/650 [==============================] - 69s 106ms/step - loss: 2.9059 - accuracy: 0.1250 - val_loss: 2.8808 - val_accuracy: 0.1281
Epoch 5/30
650/650 [==============================] - 65s 100ms/step - loss: 2.7890 - accuracy: 0.1491 - val_loss: 2.7770 - val_accuracy: 0.1636
Epoch 6/30
650/650 [==============================] - 63s 97ms/step - loss: 2.7348 - accuracy: 0.1656 - val_loss: 2.6413 - val_accuracy: 0.1833
Epoch 7/30
650/650 [==============================] - 62s 95ms/step - loss: 2.5634 - accuracy: 0.1952 - val_loss: 2.4584 - val_acc

>Setelah dilakukan training sebanyak 30 epochs, pada beberapa epochs terakhir, terjadi stuck pada peningkatan accuracy. Loss pada kedua train dan validation set juga cukup tinggi, di atas 1. 

In [ ]:
pred = model.predict_generator(generator=test_generator_df)
result_pred = []
for i in pred.argmax(-1):
    result_pred.append(upper[i])
print(result_pred)
print(accuracy_score(result_pred,y_test))

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


['A', 'E', 'L', 'N', 'S', 'D', 'G', 'J', 'W', 'M', 'N', 'Y', 'H', 'Q', 'C', 'X', 'P', 'K', 'X', 'S', 'Y', 'O', 'A', 'R', 'F', 'V', 'I', 'K']
0.6071428571428571


>Setelah di lakukan prediction dengan test data set, dapat dilihat bahwa base line CNN model memiliki tingkat akurasi yang kurang lebih sama, yaitu sekitar 60%an.

#Model Improvement Attempt 1

>Disini, saya menggunakan kombinasi output feature pada layer konvolusi. Jumlah output feature yang saya masukkan bersifat increment, dari 32 hingga 128. Hal ini karena pada layer awal, jumlah detail yang ingin dieksplor belum begitu dalam. Namun, seiring melewati layer, maka jumlah detail yang ingin dijelajah lebih banyak, sehingga output features saya tambahkan biar semakin banyak.

>Untuk setiap layer konvolusi yang terjadi, saya selipkan batch normalization. Hal ini agar process learning dapat dipercepat. Berdasarkan https://www.quora.com/Why-do-we-use-batch-normalization-in-a-convolution-neural-network-images-classification, batch normalization dapat 'meluruskan' model yang semulanya mengejar batch yang berubah-ubah.

>Pada dense layer, saya tambahkan 1 ekstra dense layer agar menambahkan jumlah komputasi yang terjadi, pada dense layer pertama, node berjumlah 128, saya samakan pada output feature pada layer konvolusi terakhir. 

>Selanjutnya, saya tambahkan dropout layers karena dirujuk dari https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/, dapat mengurangi potensi terjadinya overfitting. Hal ini karena pada proses training base model, rendahnya akurasi dan lonjakan tingkat akurasi baik pada validation data maupun test data bisa saja disebabkan oleh overfitting.



In [ ]:
keras.backend.clear_session()
model2 = Sequential()
model2.add(layers.Conv2D(32,kernel_size=5, input_shape=(64,64,3)))
model2.add(layers.Activation('relu'))
model2.add(layers.BatchNormalization())

model2.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model2.add(layers.Conv2D(64,kernel_size=5))
model2.add(layers.Activation('relu'))
model2.add(layers.BatchNormalization())

model2.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model2.add(layers.Conv2D(128,kernel_size=5))
model2.add(layers.Activation('relu'))
model2.add(layers.BatchNormalization())

model2.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model2.add(layers.Flatten())
model2.add(layers.Dense(128))
model2.add(layers.Activation('relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(64))
model2.add(layers.Activation('relu'))
model2.add(layers.Dropout(0.5))

model2.add(layers.Dense(26,activation="softmax"))

In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 32)        2432      
_________________________________________________________________
activation (Activation)      (None, 60, 60, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 60, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
activation_1 (Activation)    (None, 25, 25, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 25, 64)        2

In [ ]:
model2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
model2.fit_generator(train_generator_df, 
                    epochs=30,  # one forward/backward pass of training data
                    steps_per_epoch = X_train.shape[0]//batch_size,  # number of images comprising of one epoch
                    validation_data = valid_generator_df,  # Or validation_data=valid_generator
                    validation_steps = X_val.shape[0]//batch_size)

Epoch 1/30
650/650 [==============================] - 64s 96ms/step - loss: 3.2675 - accuracy: 0.0629 - val_loss: 3.0693 - val_accuracy: 0.0949
Epoch 2/30
650/650 [==============================] - 62s 95ms/step - loss: 3.0328 - accuracy: 0.0987 - val_loss: 2.9880 - val_accuracy: 0.1003
Epoch 3/30
650/650 [==============================] - 62s 96ms/step - loss: 2.8342 - accuracy: 0.1240 - val_loss: 2.4480 - val_accuracy: 0.2188
Epoch 4/30
650/650 [==============================] - 62s 96ms/step - loss: 2.6012 - accuracy: 0.1647 - val_loss: 2.7591 - val_accuracy: 0.1871
Epoch 5/30
650/650 [==============================] - 62s 96ms/step - loss: 2.3665 - accuracy: 0.2108 - val_loss: 2.4020 - val_accuracy: 0.2010
Epoch 6/30
650/650 [==============================] - 62s 95ms/step - loss: 2.1917 - accuracy: 0.2587 - val_loss: 2.5384 - val_accuracy: 0.2118
Epoch 7/30
650/650 [==============================] - 62s 95ms/step - loss: 2.0078 - accuracy: 0.3033 - val_loss: 1.8697 - val_accuracy:

In [ ]:
pred2 = model2.predict_generator(generator=test_generator_df)
result_pred = []
for i in pred2.argmax(-1):
    result_pred.append(upper[i])
print(result_pred)
print(accuracy_score(result_pred,y_test))

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


['A', 'E', 'L', 'N', 'S', 'D', 'G', 'I', 'W', 'N', 'B', 'X', 'H', 'Q', 'C', 'T', 'P', 'V', 'Y', 'U', 'Y', 'O', 'B', 'U', 'F', 'Z', 'J', 'K']
0.8571428571428571


>Berdasarkan hasil prediksi pada model CNN kedua, dapat dilihat bahwa terjadi peningkatan yang signifikan pada akurasi. Yang semula 60% menjadi 85% pada test data set.

>Dapat dilihat juga bahwa loss yang ada selama proses fitting untuk setiap epoch juga menurun cukup banyak. Dari yang awalnya di base model lebih dari 1 lossnya saat epoch terakhir, kali ini hanya 0.77 pada traing data dan 0.36 pada validation data

>Saya tidak menggunakan classification report karena test data set hanya masing-masing memiliki 1 data per class. Alhasil, f1-score, recall, dan precision tidak mampu menggambarkan performa model dengan semestinya.

#Model Improvement Attempt 2

>Disini, saya mencoba untuk mengurangi jumlah layer dropout, dari yang semula 2 menjadi 1. Hal ini karena berdasarkan https://stackoverflow.com/questions/46841362/where-dropout-should-be-inserted-fully-connected-layer-convolutional-layer, drop out layer harus diletakkan diantara hidden dense layer. Karena pada attempt sebelumnya juga ditambahkan sebelum output layer, saya hilangkan pada attempt ini.

In [ ]:
keras.backend.clear_session()
model3 = Sequential()
model3.add(layers.Conv2D(32,kernel_size=5, input_shape=(64,64,3)))
model3.add(layers.Activation('relu'))
model3.add(layers.BatchNormalization())

model3.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model3.add(layers.Conv2D(64,kernel_size=5))
model3.add(layers.Activation('relu'))
model3.add(layers.BatchNormalization())

model3.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model3.add(layers.Conv2D(128,kernel_size=5))
model3.add(layers.Activation('relu'))
model3.add(layers.BatchNormalization())

model3.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model3.add(layers.Flatten())
model3.add(layers.Dense(128))
model3.add(layers.Activation('relu'))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(64))
model3.add(layers.Activation('relu'))

model3.add(layers.Dense(26,activation="softmax"))
model3.summary()
model3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 32)        2432      
_________________________________________________________________
activation (Activation)      (None, 60, 60, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 60, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
activation_1 (Activation)    (None, 25, 25, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 25, 64)        2

In [ ]:
batch_size = 32
model3.fit_generator(train_generator_df, 
                    epochs=30,  # one forward/backward pass of training data
                    steps_per_epoch = X_train.shape[0]//batch_size,  # number of images comprising of one epoch
                    validation_data = valid_generator_df,  # Or validation_data=valid_generator
                    validation_steps = X_val.shape[0]//batch_size)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
650/650 [==============================] - 75s 113ms/step - loss: 2.9128 - accuracy: 0.1389 - val_loss: 2.2466 - val_accuracy: 0.2693
Epoch 2/30
650/650 [==============================] - 68s 105ms/step - loss: 2.0921 - accuracy: 0.3109 - val_loss: 1.7499 - val_accuracy: 0.4398
Epoch 3/30
650/650 [==============================] - 67s 102ms/step - loss: 1.6285 - accuracy: 0.4442 - val_loss: 1.1352 - val_accuracy: 0.6161
Epoch 4/30
650/650 [==============================] - 64s 99ms/step - loss: 1.3006 - accuracy: 0.5474 - val_loss: 1.2198 - val_accuracy: 0.5976
Epoch 5/30
650/650 [==============================] - 63s 97ms/step - loss: 1.1137 - accuracy: 0.6165 - val_loss: 0.7781 - val_accuracy: 0.7199
Epoch 6/30
650/650 [==============================] - 62s 96ms/step - loss: 0.9719 - accuracy: 0.6609 - val_loss: 0.6921 - val_accuracy: 0.7330
Epoch 7/30
650/650 [==============================] - 62s 96ms/step - loss: 0.8391 - accuracy: 0.7161 - val_loss: 0.6633 - val_accura

In [ ]:
pred3 = model3.predict_generator(generator=test_generator_df)
result_pred = []
for i in pred3.argmax(-1):
    result_pred.append(upper[i])
print(result_pred)
print(accuracy_score(result_pred,y_test))

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


['A', 'E', 'L', 'N', 'S', 'D', 'G', 'I', 'W', 'M', 'B', 'X', 'H', 'Q', 'C', 'T', 'P', 'W', 'Y', 'U', 'U', 'O', 'B', 'R', 'F', 'Z', 'J', 'K']
0.8928571428571429


>Dapat dilihat, bahwa akurasi meningkat lagi menjadi 89.28%. Berarti rujukan di atas berhasil dilaksanakan.

#Evaluasi

>Dari ketiga architecture CNN termasuk base line, didapatkan akurasi terbaik, yaitu 89.28% pada attempt ke-2 improvisasi model.

>Analisa saya berdasarkan y_test, adalah karena pada train dan valid data set, tidak terdapat class 'nothing' dan class 'space' yang terdapat pada test data set. Sehingga, kecil model belum tentu mampu mengklasifikasi image tersebut karena tidak ada pada train data set.